# Q4 - RBFNN

In [2]:
import numpy as np
from scipy.io import loadmat
from sklearn.cluster import KMeans

mat_contents = loadmat('/content/drive/MyDrive/2018A8PS1229H_NNFL/Data/data5.mat')
data = mat_contents['x']
np.random.shuffle(data)

In [3]:
def init_data():
    X = np.array(data[:2148, :-1], dtype = float)
    y = np.array(data[:2148, -1], dtype = int)
    X = (X - X.mean(axis = 0))/X.std(axis = 0)
    return X, y

def gaussian(x,center,sigma,beta):
    return np.exp(-beta * (np.linalg.norm(x - center)) ** 2)

def multi_quadric(x, center, sigma, beta):
    return ((np.linalg.norm(x - center)) ** 2 + sigma ** 2) ** 0.5

def linear(x, center, sigma, beta):
    return np.linalg.norm(x - center)

In [4]:
X_tot, y_tot = init_data()
split_num = 358
train_X = X_tot[ : 1600]
train_y = y_tot[ : 1600]
test_X = X_tot[1600 : 2148]
test_y = y_tot[1600 : 2148]

def fit_rbf(train_X, train_y, test_X, test_y):
    km = KMeans(n_clusters=550)

    y_km = km.fit_predict(train_X)
    centers = km.cluster_centers_
    labels = km.predict(train_X)

    sigma = np.zeros((len(centers), 1))
    beta = np.zeros((len(centers), 1))
    cluster_size = np.zeros((len(centers), 1))

    for i in range(len(train_X)):
        sigma[labels[i]] += np.linalg.norm(train_X[i] - centers[labels[i]])
        cluster_size[labels[i]] += 1

    sigma /= cluster_size
    beta = 1 / 2 * (sigma * sigma + 1e-6)

    H = np.zeros((len(train_X), len(centers)))

    for i in range(len(train_X)):
        for j in range(len(centers)):
            H[i, j] = linear(train_X[i], centers[j], sigma[j], beta[j])

    W = np.dot(np.linalg.pinv(H), train_y)

    #Test run
    H_test = np.zeros([len(test_X), len(centers)])
    for i in range(len(test_X)):
        for j in range(len(centers)):
            H_test[i, j] = linear(test_X[i], centers[j], sigma[j], beta[j])

    y_pred = np.dot(H_test, W)
    for i in range(len(y_pred)):
        y_pred[i] = 1 if y_pred[i]>=0.5 else 0
        
    accuracy = 0    
    for i in range(len(y_pred)):
        if y_pred[i] == test_y[i]:
            accuracy +=1
    accuracy /= len(y_pred)
    print(accuracy)
    return y_pred, accuracy

In [5]:
y_pred, _ = fit_rbf(train_X, train_y, test_X, test_y)
for i in range(len(y_pred)):
    y_pred[i] = 1 if y_pred[i] > 0.5 else 0

TP, TN, FP, FN = 0,0,0,0 

for i in range(len(test_X)):
    
    if y_pred[i] == 1 and test_y[i] == 1:
        TP += 1
    
    elif y_pred[i] == 0 and test_y[i] == 0:
        TN += 1
        
    elif y_pred[i] == 1 and test_y[i] == 0:
        FP += 1
        
    elif y_pred[i] == 0 and test_y[i] == 1:
        FN += 1
        
accuracy = (TP + TN) / (TP + TN + FP + FN)
sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)

print("accuracy = ", accuracy, "sensitivity = ", sensitivity, "specificity = ", specificity)
print(TP, FP)
print(FN, TN)
avg_acc = 0

0.958029197080292
accuracy =  0.958029197080292 sensitivity =  0.9688581314878892 specificity =  0.9459459459459459
280 14
9 245


In [6]:
# K - fold cross validation

for k in range(5):
    X = X_tot[0 : 1790]
    y = y_tot[0 : 1790]
    X_val = X_tot[1790 :]
    y_val = y_tot[1790 :]
    _, acc = fit_rbf(X, y, X_val, y_val)
    avg_acc += acc
    X_tot[: split_num] = X_val
    X_tot[split_num : ] = X
    y_tot[0 : split_num] = y_val
    y_tot[split_num : ] = y

avg_acc /= 5
print(avg_acc)

0.9497206703910615
0.9581005586592178
0.9692737430167597
0.9608938547486033
0.946927374301676
0.9569832402234637


# Q5 - Stacked Autoencoder

In [7]:
import numpy as np
from scipy.io import loadmat
from sklearn.preprocessing import normalize

#Load data, shuffle and normalize
mat_contents = loadmat('/content/drive/MyDrive/2018A8PS1229H_NNFL/Data/data5.mat')
data = mat_contents['x']
np.random.shuffle(data)

In [8]:
def init_data():
    X = np.array(data[ : , :-1], dtype = float)
    y = np.array(data[ : , -1], dtype = int)
    X = (X - X.mean(axis = 0))/X.std(axis = 0)
    return X, y

X, y = init_data()

#Hold out method of model evaluation
X_train, y_train = X[ :int(0.7 * len(X))], y[ :int(0.7 * len(X))]
X_val, y_val = X[ int(0.7 * len(X)): ], y[ int(0.7 * len(X)): ]

alpha = 0.5

#Sigmoid activation function
def sigmoid(x, derivative=False):
        if (derivative == True):
            return x * (1 - x)
        return 1 / (1 + np.exp(-x))

In [9]:
class NeuralNetwork(object):
    def __init__(self, sizes):
        
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.W = {}
        self.a = {}
        self.b = {}
        
        #Initialize Weights
        for i in range(1, self.num_layers):
            self.W[i] = np.random.randn(self.sizes[i-1], self.sizes[i])
            
        #Initialize biases
        for i in range(1, self.num_layers):
            self.b[i] = np.random.randn(self.sizes[i], 1)
        
        #Initialize activations
        for i in range(1, self.num_layers):
            self.a[i] = np.zeros([self.sizes[i], 1])
        
    #Forward pass to compute scores
    def forward_pass(self, X):
        
        self.a[0] = X
        
        for i in range(1, self.num_layers):
            self.a[i] = sigmoid(np.dot(self.W[i].T, self.a[i-1]) + self.b[i])

        return self.a[self.num_layers-1] 
    
    #Backward pass to update weights
    def backward_pass(self, X, Y, output):
        
        self.d = {}
        self.d_output = (Y - output) * sigmoid(output, derivative=True)
        self.d[self.num_layers-1] = self.d_output
        
        #Derivatives of the layers wrt loss
        for i in range(self.num_layers-1, 1, -1):
            self.d[i-1] = np.dot(self.W[i], self.d[i]) * sigmoid(self.a[i-1], derivative=True)
        
        #Updating weights
        for i in range(1, self.num_layers-1):
            self.W[i] += alpha * np.dot(self.a[i-1], self.d[i].T)
            
        #Updating biases
        for i in range(1, self.num_layers-1):
            self.b[i] += alpha * self.d[i]

    #Training helper function   
    def train(self, X, Y):
        X = np.reshape(X, (len(X), 1))
        output = self.forward_pass(X)
        self.backward_pass(X, Y, output)

    #Get weights    
    def get_W(self):
        return self.W
    
    #Load specified weights
    def load_W(self, W):
        self.W = W

    #Scores computation for given input    
    def get_a(self, x):
        x = np.reshape(x, (len(x), 1))
        self.forward_pass(x)
        return self.a
    
    #Helper function for autoencoder chaining
    def load_a(self, a):
        self.a = a

In [10]:
#Loss function
def calc_loss(NN,x ,y):
    
    loss = 0
    for i in range(len(x)):
        x_ = np.reshape(x[i], (len(x[i]), 1))
        loss += 0.5 / len(x) * np.sum((y[i] - NN.forward_pass(x_)) ** 2)
    
    return loss

In [11]:
#Network initialization
autoencoder1 = NeuralNetwork([72, 60, 72])
autoencoder2 = NeuralNetwork([60,40,60])
autoencoder3 = NeuralNetwork([40, 30, 40])
NN = NeuralNetwork([72,60,40,30, 1])

#Autoencoder 1 pretraining
for i in range(200):
    for j, row in enumerate(X_train):
        row = np.reshape(row, (72,1))
        autoencoder1.train(row, row)
        
    loss = calc_loss(autoencoder1, X_train, X_train)
    print("Epoch {}, Loss {}".format(i, loss))
    
#Scores computation for autoencoder 1
autoencoder2_input = []

for row in X_train:
    autoencoder2_input.append(autoencoder1.get_a(row)[1])

autoencoder2_input = np.array(autoencoder2_input)


#Autoencoder 2 pretraining
for i in range(200):
    for j, row in enumerate(autoencoder2_input):
        row = np.reshape(row, (60,1))
        autoencoder2.train(row, row)
        
    loss = calc_loss(autoencoder2, autoencoder2_input, autoencoder2_input)
    print("Epoch {}, Loss {}".format(i, loss))


#Scores computation for autoencoder 2
autoencoder3_input = []

for row in autoencoder2_input:
    autoencoder3_input.append(autoencoder2.get_a(row)[1])

autoencoder3_input = np.array(autoencoder3_input)

#Autoencoder 3 pretraining
for i in range(200):
    for j, row in enumerate(autoencoder3_input):
        row = np.reshape(row, (40,1))
        autoencoder3.train(row, row)
        
    loss = calc_loss(autoencoder3, autoencoder3_input, autoencoder3_input)
    print("Epoch {}, Loss {}".format(i, loss))

#Final network weight initialization
W1 = autoencoder1.get_W()[1]
W2 = autoencoder2.get_W()[1]
W3 = autoencoder3.get_W()[1]
W_final = {}
W_final[1] = W1
W_final[2] = W2
W_final[3] = W3
W_final[4] = np.random.randn(30, 1)
NN.load_W(W_final)

#Training loop
for i in range(500):
    print("Epoch: ", i)

    for j in range(len(X_train)):
        NN.train(X_train[j], y_train[j])

TP,TN,FP,FN = 0,0,0,0

for i in range(len(X_val)):

    x = np.reshape(X_val[i], (len(X_val[i]), 1))
    x = NN.forward_pass(x)
    p = 0 if x[0] < 0.5 else 1

    if p == 1 and y_val[i] == 1:
        TP += 1
    elif p == 0 and y_val[i] == 0:
        TN += 1
    elif p == 1 and y_val[i] == 0:
        FP += 1
    elif p == 0 and y_val[i] == 1:
        FN += 1

print(TP, FP)
print(FN, TN)

accuracy = (TP + TN) / (TP + TN + FP + FN)
sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)

print("accuracy = ", accuracy, "sensitivity = ", sensitivity, "specificity = ", specificity)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in exp


Epoch 0, Loss 3111.9183457398008
Epoch 1, Loss 3073.356819501463
Epoch 2, Loss 3055.3164064178104
Epoch 3, Loss 3043.6610746389683
Epoch 4, Loss 3027.6370621306073
Epoch 5, Loss 3033.9044169372537
Epoch 6, Loss 3031.119042934751
Epoch 7, Loss 3024.6214952993078
Epoch 8, Loss 3026.1328658236935
Epoch 9, Loss 3020.918916457581
Epoch 10, Loss 3019.3618279588263
Epoch 11, Loss 3017.5086773918356
Epoch 12, Loss 3019.3579187095984
Epoch 13, Loss 3016.5892190301192
Epoch 14, Loss 3024.7139394218616
Epoch 15, Loss 3021.228438967541
Epoch 16, Loss 3023.45836490839
Epoch 17, Loss 3021.706364820135
Epoch 18, Loss 3018.71839333829
Epoch 19, Loss 3019.8763625720585
Epoch 20, Loss 3017.110749625585
Epoch 21, Loss 3021.1345664197765
Epoch 22, Loss 3017.965975351277
Epoch 23, Loss 3023.398653109415
Epoch 24, Loss 3019.99152442701
Epoch 25, Loss 3017.772690291936
Epoch 26, Loss 3021.323439296316
Epoch 27, Loss 3013.714590338511
Epoch 28, Loss 3017.9868374327543
Epoch 29, Loss 3015.98503621487
Epoch 30,